# Derivadas superiores

Hasta ahora hemos visto que, usando diferenciación automática, podemos calcular la derivada de funciones de una variable esencialmente con un error del orden del epsilon de la máquina.

La pregunta que abordaremos ahora, es cómo hacer para calcular la segunda derivada, y derivadas de orden superior.

Una posibilidad, específica para el caso de la segunda derivada, es proceder como en el caso anterior, es decir, definir una *terna ordenada* donde la primer componente es el valor de la función en $x_0$, i.e., $f(x_0)$, el de la segunda es el valor de la primer derivada $f'(x_0)$, y la tercer componente tiene el valor de la segunda derivada, $f^{(2)}(x_0) = f''(x_0)$. 


Con esta definición, las operaciones aritméticas vienen dadas por:

\begin{eqnarray}
\vec{u} + \vec{v} & = & (u + v, \quad u'+ v', \quad u''+v''),\\
\vec{u} - \vec{v} & = & (u - v, \quad u'- v', \quad u''-v''),\\
\vec{u} \times \vec{v} & = & (u v, \quad u v' + u' v, \quad u v'' + 2 u' v' + u'' v),\\
\frac{\vec{u}}{\vec{v}} & = & \Big( \frac{u}{v}, \quad \frac{u'-( u/v) v'}{v}, \quad 
\frac{u'' - 2 (u/v)' v' - (u/v)v'' }{v}\Big).\\
\end{eqnarray}

Claramente, este proceso es muy ineficiente para derivadas de orden aún más alto, dado que las expresiones se complican y es fácil cometer errores.

# Series de Taylor

El punto importante a recordar, es que las derivadas de orden superior de una función $f(x)$ en un punto $x_0$ están contenidas en el desarrollo de Taylor de la función entorno a ese punto. La suposición importante en esto es que $f(x)$ es suficientemente suave; por simplicidad supondremos que $f(x)$ es ${\cal C}^\infty$ y que estamos suficientemente cerca del punto $x_0$, i.e., $|x-x_0|\ll 1$. 


La serie de Taylor de $f(x)$ viene dada por

\begin{eqnarray}
f(x) & = & f(x_0) + f^{(1)}(x_0) (x-x_0) + \frac{f^{(2)}(x_0)}{2!} (x-x_0)^2 + \dots + \frac{f^{(k)}(x_0)}{k!} (x-x_0)^k + \dots,\\
& = & f_{[0]} + f_{[1]} (x-x_0) + f_{[2]} (x-x_0)^2 + \dots + f_{[k]} (x-x_0)^k + \dots,\\
\end{eqnarray}

donde los coeficientes *normalizados* de Taylor $f_{[k]}$ que aparecen en la segunda línea de la ecuación anterior se definen como

\begin{equation}
f_{[k]} = \frac{f^{(k)}(x_0)}{k!} = \frac{1}{k!}\frac{{\rm d}^k f}{{\rm d}x^k}(x_0).
\end{equation}



Vale la pena **enfatizar** que la expresión anterior es exacta en tanto que la serie **no** sea truncada. En el caso de que la serie truncada a orden k, el [teorema de Taylor](https://en.wikipedia.org/wiki/Taylor%27s_theorem) asegura que el residuo (error del truncamiento) se puede escribir como:

\begin{equation}
{\cal R_{k}} = \frac{f^{(k+1)}\,(\xi)}{(k+1)!} (x-x_0)^{k+1},
\end{equation}

donde $\xi$ es un punto que pertenece al interval $[x_0,x]$.


Si la serie es truncada, la aproximación es un polinomio de orden $k$ (grado máximo es $k$) en $x$. Dado que los polinomios en una variable están definidos por $k+1$ coeficientes, entonces pueden ser mapeados a vectores en $\mathbb{R}^{k+1}$. 

Las operaciones aritméticas, en este caso, vienen dadas por:

\begin{eqnarray}
(f+g)_{[k]} & = & f_{[k]} + g_{[k]} ,\\
(f-g)_{[k]} & = & f_{[k]} - g_{[k]} ,\\
(f \cdot g)_{[k]} & = & \sum_{i=0}^k f_{[i]} \,g_{[k-i]} \, ,\\
\Big(\frac{f}{g}\Big)_{[k]} & = & \frac{1}{g_{[0]}}
\Big( f_{[k]} - \sum_{i=0}^{k-1} \big(\frac{f}{g}\big)_{[i]} \, g_{[k-i]} \Big) . \\
\end{eqnarray}

### Ejercicio

Implementen una nueva estructura paramétrica (`type`) que defina el tipo `Taylor`, donde el parámetro debe ser un subtipo de `Number`. Definan métodos que implementen las operaciones aritméticas básicas (`+`, `-`, `*`, `/`) y la igualdad (`==`). Esto deberá ser incluido en un módulo.

Incluyan pruebas (en el archivo "runtests.jl") para cada uno de los métodos que implementen.


In [2]:
"""Definición de polinomios de Taylor, donde
...
"""
type Taylor{T<:Number} <: Number
    coffs :: Array{T,1}
    order :: Int
    
    function Taylor(coffs::Array{T,1}, order::Int)
        lencoef = length(coffs)
        order = max(order, lencoef-1)
        order == lencoef-1 && return new(coffs, order)
        resize!(coffs, order+1)
        for i = lencoef+1:order+1
            coffs[i] = zero(T)
        end
        new(coffs, order)
    end
end

In [3]:
Taylor{T<:Number}(x::Taylor{T}, order::Int) = Taylor{T}(x.coffs, order)
Taylor{T<:Number}(x::Taylor{T}) = x
Taylor{T<:Number}(coffs::Array{T,1}, order::Int) = Taylor{T}(coffs, order)
Taylor{T<:Number}(coffs::Array{T,1}) = Taylor{T}(coffs, length(coffs)-1)
Taylor{T<:Number}(x::T, order::Int) = Taylor{T}([x], order)
Taylor{T<:Number}(x::T) = Taylor{T}([x], 0)
Taylor{T<:Irrational}(coffs::Array{T,1}) =  Taylor(convert(Array{Float64,1}, coffs))
Taylor{T<:Irrational}(x::T) = Taylor(Array{Float64}([x]), 0)

Taylor{T<:Number}

In [4]:
import Base: print,show
function show(io::IO,x::Taylor)
    
    const simb = [c for c in "₀₁₂₃₄₅₆₇₈₉"]
    function sup(n::Int)
        dig = reverse(digits(n))
        join([simb[i+1] for i in dig])
    end
    
    n=length(x.coffs)-1
    
    for i = 1:n
        print(io,x.coffs[i],"ₜ",sup(i-1)," + ")
    end
    print(io,x.coffs[n+1],"ₜ",sup(n))
end
print(io::IO,x::Taylor)=show(io,x)

print (generic function with 19 methods)

In [5]:
Taylor([1,2,3,4,5,62])

1ₜ₀ + 2ₜ₁ + 3ₜ₂ + 4ₜ₃ + 5ₜ₄ + 62ₜ₅

In [ ]:
import Base: +, -, *, /, ^, == 
for f in (:+, :-)
    @eval begin
        function $f(A::Taylor,B::Taylor)
            a=zeros(Number,max(length(A.coffs),length(B.coffs)))
            for i=1:length(A.coffs)
                a[i]=A.coffs[i]
            end
            for i=1:length(B.coffs)
                a[i]=$f(a[i],B.coffs[i])
            end
            Taylor(a)
        end
        $f(x::Taylor,y::Number)=$f(x,Taylor(y))
        $f(y::Number,x::Taylor)=$f(x,Taylor(y))
    end
end
function *(A::Taylor,B::Taylor)
    a=zeros(Number,length(A.coffs)+length(B.coffs)-1)
    for k=1:length(a)
        for i=1:k
            if(i<=length(A.coffs) && k-i+1<=length(B.coffs))
                a[k]+=A.coffs[i]*B.coffs[k-i+1]
            end
        end
    end
    Taylor(a)
end
*(x::Taylor,y::Number)=x*Taylor(y)
*(y::Number,x::Taylor)=x*Taylor(y) 

function /(A::Taylor,B::Taylor)
    a=zeros(Number,max(length(A.coffs),length(B.coffs)))
    a[1]=A.coffs[1]/B.coffs[1]
    for k=2:length(a)
        for i=1:k-1
            if(k-i+1<=length(B.coffs))
                a[k]+=a[i]*B.coffs[k-i+1]
            end
        end
        if(k<=length(A.coffs))
            a[k]=A.coffs[k]-a[k]
        end
        if(B.coffs[1]!=0)
            a[k]=a[k]/B.coffs[1]
        else
            error("division by zero")
        end
    end
    Taylor(a)
end
/(x::Taylor,y::Number)=x/Taylor(y)
/(y::Number,x::Taylor)=x/Taylor(y)

function ==(A::Taylor,B::Taylor)
    R=true;
    if(length(A.coffs)!=length(B.coffs))
        a=zeros(Number,max(length(A.coffs),length(B.coffs)))
        b=zeros(Number,max(length(A.coffs),length(B.coffs)))
        for i=1:length(A.coffs)
            a[i]=A.coffs[i]
        end
        for i=1:length(B.coffs)
            b[i]=B.coffs[i]
        end
        for i=1:length(a)
            if(a[i]!=b[i])
                    R=false
            end
        end
    else
        for i=1:length(A.coffs)
            if(A.coffs[i]!=B.coffs[i])
                    R=false
            end
        end
    end
    R
end

In [7]:
a=Taylor([15,3])
b=Taylor([4,5])
c=2
print("a:\t",a,"\n")
print("b:\t",b,"\n")
print("c:\t",c,"\n\n")

print("a+b:\t",a+b,"\n")
print("a+c:\t",a+c,"\n")

print("a-b:\t",a-b,"\n")
print("a-c:\t",a-c,"\n")

print("a*b:\t",a*b,"\n")
print("a*c:\t",a*c,"\n")

print("a/b:\t",a/b,"\n")
print("a/c:\t",a/c,"\n")

print("a==b:\t",a==b,"\n")
print("a==a:\t",a==a,"\n")

a:	15ₜ₀ + 3ₜ₁
b:	4ₜ₀ + 5ₜ₁
c:	2

a+b:	19ₜ₀ + 8ₜ₁
a+c:	17ₜ₀ + 3ₜ₁
a-b:	11ₜ₀ + -2ₜ₁
a-c:	13ₜ₀ + 3ₜ₁
a*b:	60ₜ₀ + 87ₜ₁ + 15ₜ₂
a*c:	30ₜ₀ + 6ₜ₁
a/b:	3.75ₜ₀ + -3.9375ₜ₁
a/c:	7.5ₜ₀ + 1.5ₜ₁
a==b:	false
a==a:	true


In [23]:
# Muestren que su código funciona con tests adecuados; para los detalles ver 
# http://julia.readthedocs.org/en/release-0.4/stdlib/test/
using Base.Test
A=(0,1,7,0.0,1.0,7.0,pi,-1,-7)
for (a in A)
    for(b in A)
        d=Taylor([a,b]);
        c=a
        if(a!=0)
            @test (d+d-d*d/d)==d
            @test (d+a-a*a/a)==d
            @test (c+d-d*d/d)==Taylor(c)
        else
            @test ((d+d-d)*d)==d*d
            @test ((d+a-a)*a)==d*a
            @test ((c+d-d)*d)==Taylor(c*d)
        end
    end
end

# Funciones de polinomios

El siguiente punto, es cómo definir funciones de polinomios. 

Como veremos aquí, esto se basará en plantear una ecuación diferencial apropiada, cuya solución es, precisamente, la expresión que estamos buscando. Este punto es *importante*, y muestra que hay una conexión importante con la solución de ecuaciones diferenciales.

Como ejemplo consideraremos la función

\begin{equation}
E(x) = \exp\big(g(x)\big),
\end{equation}

donde 

\begin{equation}
g(x) = \sum_{k=0}^\infty g_{[k]} (x-x_0)^k
\end{equation}

está escrita como una serie de Taylor (¡exacta!) alrededor de $x_0$. 


El primer punto, es que escribiremos a $E(x)$ como una serie de Taylor alrededor de $x_0$, es decir,

\begin{equation}
E(x) = \sum_{k=0}^\infty E_{[k]} (x-x_0)^k.
\end{equation}

El objetivo es determinar $E_{[k]}$ para *toda* $k$.

Dado que $E(x)$ esun polinomio en $x$, su derivada viene dada por

\begin{equation}
\frac{{\rm d} E(x)}{{\rm d}x} = \sum_{k=1}^\infty k E_{[k]}\, (x-x_0)^{k-1} .
\end{equation}

Por otra parte, la derivada de $E(x)$ en términos de $g(x)$ está dada por

\begin{equation}
\frac{{\rm d} E(x)}{{\rm d}x} = \exp\big(g(x)\big) \frac{{\rm d} g(x)}{{\rm d}x} = E(x) \frac{{\rm d} g(x)}{{\rm d}x},
\end{equation}

donde del lado derecho aparece la derivada de $g(x)$. Ya que $g(x)$ *también* está escrita en forma polinomial, su derivada es

\begin{equation}
\frac{{\rm d} g(x)}{{\rm d}x} = \sum_{k=1}^\infty k g_{[k]}\, (x-x_0)^{k-1} .
\end{equation}


Tenemos, entonces, todo lo que requerimos para escribir el lado derecho de la ecuación diferencial y explotar la aritmética de polinomios. 

\begin{eqnarray}
E(x) \frac{{\rm d} g(x)}{{\rm d}x}& = & 
\Big[ \sum_{k=0}^\infty E_{[k]} (x-x_0)^k \Big]
\Big[ \sum_{j=1}^\infty j g_{[j]} (x-x_0)^{j-1}\Big] \\
 & = & \sum_{k=1}^\infty \Big[ \sum_{j=0}^k j g_{[j]} E_{[k-j]} \; \Big] (x-x_0)^{k-1} .\\
\end{eqnarray}

La segunda línea se obtiene reordenando los términos al fijar la potencia de $(x-x_0)$, esto es, $k+j$ se toma como un nuevo índice ($k$), y el nuevo índice $j$ describe el índice del producto de los polinomios. (La potencia se deja de la forma $k-1$ ya que el lado izquierdo de la ecuación aparece así.)

Igualando con el lado izquierdo de la ecuación diferencial, que sólo involucra a la derivada de $E(x)$, tenemos que se debe cumplir

\begin{equation}
E_{[k]} = \frac{1}{k} \sum_{j=0}^k j g_{[j]} \, E_{[k-j]} = 
\frac{1}{k} \sum_{j=0}^{k} (k-j) g_{[k-j]} \, E_{[j]}, \qquad k=1,2,\dots,
\end{equation}

incluyendo *la condición inicial*

\begin{equation}
E_{[0]} = \exp\big(g(x_0)\big).
\end{equation}

Estas relaciones *de recurrencia* permiten calcular $\exp\big(g(x)\big)$, para cualquier polinomio $g(x)$.

Para el caso concreto $g(x) = x$ alrededor de $x_0=0$, donde tenemos $g_{[j]} = \delta_{j,1}$, obtenemos

\begin{eqnarray}
E_{[0]} & = & 1,\\
E_{[k]} & = & \frac{1}{k} E_{[k-1]} = \frac{1}{k(k-1)} E_{[k-2]} = \dots = \frac{1}{k!} E_{[0]} = \frac{1}{k!}\ ,
\end{eqnarray}

que es el resultado bien conocido.

### Ejercicio

Obtengan las relaciones de recurrencia para las funciones $L(x) = \log\big(g(x)\big)$, $P_\alpha(x) = \big(g(x)\big)^\alpha$, $S(x) = \sin\big(g(x)\big)$, $C(x) = \cos\big(g(x)\big)$ usando el mismo procedimiento que arriba. Implementen métodos adecuados para estas funciones en el módulo, actuando sobre estructuras `Taylor` e incluyan pruebas.

In [ ]:
import Base: exp,log,^,sin,cos
function exp(A::Taylor)
    a=zeros(Number,length(A.coffs))
    a[1]=exp(A.coffs[1])
    for k=2:length(a)
        for j=1:k-1
            a[k]+=(k-j)*A.coffs[k-j+1]*a[j]
        end
        a[k]=a[k]/(k-1)
    end
    Taylor(a)
end
function log(A::Taylor)
    d=zeros(Number,length(A.coffs)-1)
    for k=1:length(d)
        d[k]=A.coffs[k+1]*(k)
    end
    a=Taylor(d)/A
    r=zeros(Number,length(A.coffs))
    for i=2:length(a.coffs-1)
        r[i]=a.coffs[i-1]/(i-1)
    end
    r[1]=log(A.coffs[1])
    Taylor(r)
end
^(A::Taylor,x::Integer)=exp(x*log(A))
^(A::Taylor,x::Real)=exp(x*log(A))
^(A::Taylor,x::Rational)=exp(x*log(A))
function sin(A::Taylor)
    a=zeros(Number,length(A.coffs))
    r=Taylor(a)
    for n=0:length(a)+1
        r+=(((-1)^n)/(factorial((2*n)+1)))*(A^((2*n)+1))
    end
    r
end
function cos(A::Taylor)
    a=zeros(Number,length(A.coffs))
    r=Taylor(a)
    for n=0:length(a)+1
        r+=(((-1)^n)/(factorial(2*n)))*(A^(2*n))
    end
    r
end

In [10]:
a=Taylor([15,3])

exp,log,^,sin,cos
print("a:\t",a,"\n")

print("exp(a):\t",exp(a),"\n")
print("log(a):\t",log(a),"\n")
print("a^2:\t",a^2,"\n")
print("sin(a):\t",sin(a),"\n")
print("cos(a):\t",cos(a),"\n")

a:	15ₜ₀ + 3ₜ₁
exp(a):	3.2690173724721107e6ₜ₀ + 9.807052117416332e6ₜ₁
log(a):	2.70805020110221ₜ₀ + 0.2ₜ₁
a^2:	225.0ₜ₀ + 90.0ₜ₁
sin(a):	-28120.04464285712ₜ₀ + -41467.312499999985ₜ₁
cos(a):	-13822.437499999985ₜ₀ + -17341.874999999985ₜ₁


**NOTA** Los ejercicios de este notebook constituyen el contenido de la Tarea6.